In [1]:
import cv2
from itertools import product

# Function to detect faces and return the number of detected faces
def detect_faces(image, scaleFactor, minNeighbors, minSize, cascade_file):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) for enhanced contrast
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    
    face_cascade = cv2.CascadeClassifier(cascade_file)
    
    # Use a dictionary for named arguments to ensure flags are set correctly
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=scaleFactor,
        minNeighbors=minNeighbors,
        minSize=minSize,
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    return faces

# List of cascade classifier file paths
cascade_files = [
    'haarcascade_frontalface_default.xml',
    'haarcascade_frontalface_alt.xml',
    'haarcascade_frontalface_alt2.xml',
    'haarcascade_frontalface_alt_tree.xml'
]

# List of paths to your images (webp and other formats)
image_paths = [
    'CSC515-crowd1.webp',
    'CSC515-crowd2.jpeg',
    'CSC515-crowd3.jpeg'
]

for image_path in image_paths:
    original_image = cv2.imread(image_path)

    if original_image is not None:
        best_params = None
        best_face_count = 0

        # Define the parameter grid for scaleFactor, minNeighbors, and minSize
        param_grid = {
            'scaleFactor': [1.1, 1.2, 1.3],
            'minNeighbors': [3, 4, 5],
            'minSize': [(20, 20), (30, 30), (40, 40)]
        }

        # Try different cascade classifiers and parameter combinations
        for cascade_file in cascade_files:
            for params in product(param_grid['scaleFactor'], param_grid['minNeighbors'], param_grid['minSize']):
                faces = detect_faces(original_image, *params, cascade_file)
                if len(faces) > best_face_count:
                    best_face_count = len(faces)
                    best_params = params

        # Draw bounding boxes on the original image based on the best parameters
        if best_params:
            scaleFactor, minNeighbors, minSize = best_params
            face_cascade = cv2.CascadeClassifier(cascade_files[0])  # Use the first cascade file for detection
            faces = face_cascade.detectMultiScale(
                original_image,
                scaleFactor=scaleFactor,
                minNeighbors=minNeighbors,
                minSize=minSize,
                flags=cv2.CASCADE_SCALE_IMAGE
            )
            for (x, y, w, h) in faces:
                cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            cv2.imshow('Detected Faces - ' + image_path, original_image)
            cv2.waitKey(0)  # Wait indefinitely until a key is pressed
            cv2.destroyAllWindows()
        else:
            print("No faces detected in", image_path)
    else:
        print("Error loading the image:", image_path)
